In [6]:
#Ignoring TF warning messages
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" #21BAI1210

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from torchvision import models

In [8]:
# Define transformation for the CIFAR-10 images
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Inception v3 requires input size (299, 299)
    transforms.ToTensor(),
])

# Load the CIFAR-10 dataset
full_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

# Use only a small subset for demonstration (adjust as needed)
subset_size = 1000
subset_dataset, _ = random_split(full_dataset, [subset_size, len(full_dataset) - subset_size])

# DataLoader
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [9]:
# Function to train and evaluate a model
def train_and_evaluate(model, name):
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Train the model
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Evaluate the model on the training set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        train_accuracy = correct / total
        print(f'{name} - Epoch [{epoch + 1}/{num_epochs}], Training Accuracy: {train_accuracy * 100:.2f}%')

In [10]:
# Train and evaluate VGG16
vgg16 = models.vgg16(weights=True)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 10)
train_and_evaluate(vgg16, "VGG16")

/home/yaduk/anaconda3/envs/deepl/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG16 - Epoch [1/5], Training Accuracy: 9.90%
VGG16 - Epoch [2/5], Training Accuracy: 9.90%
VGG16 - Epoch [3/5], Training Accuracy: 10.90%
VGG16 - Epoch [4/5], Training Accuracy: 10.90%
VGG16 - Epoch [5/5], Training Accuracy: 10.90%


In [11]:
# Train and evaluate GoogLeNet (Inception v3)
googlenet = models.inception_v3(weights=True, aux_logits=True)  # aux_logits=True to include auxiliary classifier
googlenet.aux_logits = False  # Remove the auxiliary classifier
googlenet.fc = nn.Linear(googlenet.fc.in_features, 10)
train_and_evaluate(googlenet, "GoogLeNet") #21BAI1210

/home/yaduk/anaconda3/envs/deepl/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/yaduk/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [01:44<00:00, 1.05MB/s] 


GoogLeNet - Epoch [1/5], Training Accuracy: 39.60%
GoogLeNet - Epoch [2/5], Training Accuracy: 53.10%
GoogLeNet - Epoch [3/5], Training Accuracy: 73.80%
GoogLeNet - Epoch [4/5], Training Accuracy: 79.30%
GoogLeNet - Epoch [5/5], Training Accuracy: 76.50%
